In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sklearn

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from pathlib import Path
from IPython.display import Image, display, Video, HTML
from ipywidgets import interact, widgets

from signlens.params import *
from signlens.preprocessing import data, preprocess
from signlens.model import workflow
from signlens.model.model_architecture import initialize_model, compile_model, train_model, evaluate_model
from signlens.model import model_utils

from utils import plot_landmarks


2024-03-26 17:10:39.601592: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 17:10:41.138636: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
pq_files = data.load_data_subset_csv().file_path

Loading data subset from train_train.csv
    ℹ️ Filtered sequences with missing frames. Size reduced from 77228 to 77228 (100.00%)
    ℹ️ Filtered on n_frames = 100. Size reduced from 77228 to 68935 (89.26%)
    ℹ️ Filtered on n_classes = 30. Size reduced from 68935 to 828 (1.20%)
✅ Loaded 828 rows (1.07% of the original 77228 rows) from the dataset.


In [3]:
pq_path = pq_files.iloc[0]
pq_path

'/home/bfrisque/code/benoitfrisque/signlens/raw_data/asl-signs/train_landmark_files_noface/18796/2361122045.parquet'

In [4]:
data.load_relevant_data_subset(pq_path)

array([[[        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [ 0.4625222 ,  0.3789413 ],
        [ 0.5050153 ,  0.33047566],
        [ 0.5255081 ,  0.332459  ],
        [ 0.54508764,  0.3346208 ],
        [ 0.43392685,  0.32768947],
        [ 0.4067065 ,  0.32678083],
        [ 0.38307142,  0.326

In [5]:
data_columns = ['x', 'y', 'z', 'type','landmark_index']  # Include the 'type' column

data = pd.read_parquet(pq_path, columns=data_columns)
data

,x,y,z,type,landmark_index
468,NaN,NaN,NaN,left_hand,0
469,NaN,NaN,NaN,left_hand,1
470,NaN,NaN,NaN,left_hand,2
471,NaN,NaN,NaN,left_hand,3
472,NaN,NaN,NaN,left_hand,4
...,...,...,...,...,...
3253,0.236596,0.624814,-0.076208,right_hand,16
3254,0.076827,0.624981,-0.076172,right_hand,17
3255,0.160885,0.609119,-0.097609,right_hand,18
3256,0.193975,0.626310,-0.085724,right_hand,19


In [6]:
json_path = os.path.join(LANDMARKS_VIDEO_DIR, '07070_landmarks.json')
json_data = pd.read_json(json_path)

In [7]:
json_data

,frame_number,pose,left_hand,right_hand
0,0,"[{'landmark_index': 0, 'x': 0.520770907402038,...","[{'landmark_index': 0, 'x': None, 'y': None, '...","[{'landmark_index': 0, 'x': None, 'y': None, '..."
1,1,"[{'landmark_index': 0, 'x': 0.5283789634704591...","[{'landmark_index': 0, 'x': None, 'y': None, '...","[{'landmark_index': 0, 'x': None, 'y': None, '..."
2,2,"[{'landmark_index': 0, 'x': 0.5320750474929811...","[{'landmark_index': 0, 'x': None, 'y': None, '...","[{'landmark_index': 0, 'x': None, 'y': None, '..."
3,3,"[{'landmark_index': 0, 'x': 0.5344632267951961...","[{'landmark_index': 0, 'x': None, 'y': None, '...","[{'landmark_index': 0, 'x': None, 'y': None, '..."
4,4,"[{'landmark_index': 0, 'x': 0.536107540130615,...","[{'landmark_index': 0, 'x': None, 'y': None, '...","[{'landmark_index': 0, 'x': None, 'y': None, '..."
...,...,...,...,...
81,81,"[{'landmark_index': 0, 'x': 0.526905357837677,...","[{'landmark_index': 0, 'x': None, 'y': None, '...","[{'landmark_index': 0, 'x': None, 'y': None, '..."
82,82,"[{'landmark_index': 0, 'x': 0.527315199375152,...","[{'landmark_index': 0, 'x': None, 'y': None, '...","[{'landmark_index': 0, 'x': None, 'y': None, '..."
83,83,"[{'landmark_index': 0, 'x': 0.527254581451416,...","[{'landmark_index': 0, 'x': None, 'y': None, '...","[{'landmark_index': 0, 'x': None, 'y': None, '..."
84,84,"[{'landmark_index': 0, 'x': 0.5247585177421571...","[{'landmark_index': 0, 'x': None, 'y': None, '...","[{'landmark_index': 0, 'x': None, 'y': None, '..."


In [12]:
import json

def convert_landmarks_json_file_to_df(json_path):
    with open(json_path, 'r') as file:
        json_data = json.load(file)
        json_df = convert_landmarks_json_data_to_df(json_data)
        return json_df

def convert_landmarks_json_data_to_df(json_data):
    # Initialize an empty list to store the converted data
    converted_data = []

    # Iterate over each frame in the JSON data
    for frame in json_data:
        # Extract pose landmarks
        pose_landmarks = frame['pose']
        for landmark in pose_landmarks:
            converted_data.append({
                'x': landmark['x'],
                'y': landmark['y'],
                'z': landmark['z'],
                'type': 'pose',
                'landmark_index': landmark['landmark_index']
            })

        # Extract left hand landmarks
        left_hand_landmarks = frame['left_hand']
        for landmark in left_hand_landmarks:
            converted_data.append({
                'x': landmark['x'],
                'y': landmark['y'],
                'z': landmark['z'],
                'type': 'left_hand',
                'landmark_index': landmark['landmark_index']
            })

        # Extract right hand landmarks
        right_hand_landmarks = frame['right_hand']
        for landmark in right_hand_landmarks:
            converted_data.append({
                'x': landmark['x'],
                'y': landmark['y'],
                'z': landmark['z'],
                'type': 'right_hand',
                'landmark_index': landmark['landmark_index']
            })

    # Create a DataFrame from the converted data
    df = pd.DataFrame(converted_data)

    return df


In [13]:
convert_landmarks_json_file_to_df(json_path)

,x,y,z,type,landmark_index
0,0.520771,0.282746,-1.179220,pose,0
1,0.550389,0.246610,-1.133541,pose,1
2,0.565520,0.249983,-1.133734,pose,2
3,0.579948,0.254050,-1.133764,pose,3
4,0.499756,0.240471,-1.139189,pose,4
...,...,...,...,...,...
6445,NaN,NaN,NaN,right_hand,16
6446,NaN,NaN,NaN,right_hand,17
6447,NaN,NaN,NaN,right_hand,18
6448,NaN,NaN,NaN,right_hand,19
